In [ ]:
print("hello world")

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
 #from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI


In [ ]:

llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # e.g., "gpt-4o", "gpt-4o-mini", "omni-gpt-4", "omni-gpt-4-mini"
    # stream_usage=True,
    temperature=0.0,
    max_tokens=2000,
    # timeout=None,
    #reasoning_effort="low",
    # max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instead of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)   

In [ ]:
#!/usr/bin/env python3
"""
LangGraph: COMPLEX RIVER CROSSING PUZZLE
SAME HARD TASK → 4 AGENTS FAIL/SUCCEED DIFFERENTLY
Task: Cabbage/Goat/Lion puzzle (ReAct+Reflection FAIL, Reflexion+Tree SUCCEED)
"""

import os
from typing import Annotated, Dict, List, Literal
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI



# # COMPLEX PUZZLE (Designed to TRICK simple agents)
# PUZZLE = """Suppose I have a cabbage, a goat and a lion, and I need to get them across a river. 
# I have a boat that can only carry myself and a single other item. 
# I am not allowed to leave the cabbage and lion alone together, 
# and I am not allowed to leave the lion and goat alone together. 
# How can I safely get all three across?"""

# CORRECT_SOLUTION = """
# 1. Take lion across (leave goat+cabbage)
# 2. Return alone  
# 3. Take cabbage across (leave goat alone)  
# 4. Bring lion back (leave cabbage alone)
# 5. Take goat across (leave lion alone)
# 6. Return alone
# 7. Take lion across ✓
#"""

PUZZLE = """pick two random 5 digit numbers and multiply them together"""

In [ ]:

# =============================================================================
# 🎮 1. REAct AGENT (FAILS - Single pass can't solve complex planning)
# =============================================================================
def react_node(state: Dict) -> Dict:
    prompt = f"""ReAct Agent (1 step): {PUZZLE}

Thought: [brief reasoning]
Action: [solution sequence]"""
    msg = llm.invoke([HumanMessage(content=prompt)])
    result = msg.content
    # ReAct FAILS: Misses full sequence
    #return {"result": result, "status": "❌ FAILED (incomplete)", "messages": [AIMessage("🎮 ReAct: 1 step FAIL")]}
    return result

react_graph = StateGraph(dict)
react_graph.add_node("react", react_node)
react_graph.add_edge(START, "react")
react_graph.add_edge("react", END)
REACT_AGENT = react_graph.compile()



In [ ]:
from IPython.display import Image, display
display(Image(REACT_AGENT.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
print("\n🎮 1. REAct Agent")
print("-" * 40)
react_result = REACT_AGENT.invoke({"task": PUZZLE})
print(f"   {react_result}")
#print(f"   Output: {react_result['result']}...")
    

In [ ]:
# =============================================================================
# 🪞 2. REFLECTION AGENT (FAILS - 1 reflection not enough for 7-step puzzle)
# =============================================================================
def reflection_generate(state: Dict) -> Dict:
    msg = llm.invoke([HumanMessage(content=f"Solve: {PUZZLE}")])
    print(msg.content)
    return {"attempt": msg.content}

def reflection_critique(state: Dict) -> Dict:
    critique = llm.invoke([HumanMessage(f"CRITIQUE: {PUZZLE}\nAttempt: {state['attempt']}\ is the solution correct?")])
    return {"critique": critique.content}

def reflection_fix(state: Dict) -> Dict:
    # Still fails - 1 reflection can't solve complex constraint puzzle
    prompt = f"Fix: {PUZZLE}\nOld: \nCritique: {state['critique']}\nFIXED:"
    msg = llm.invoke([HumanMessage(content=prompt)])
    return msg

reflection_graph = StateGraph(dict)
reflection_graph.add_node("generate", reflection_generate)
reflection_graph.add_node("critique", reflection_critique) 
reflection_graph.add_node("fix", reflection_fix)
reflection_graph.add_edge(START, "generate")
reflection_graph.add_edge("generate", "critique")
reflection_graph.add_edge("critique", "fix")
reflection_graph.add_edge("fix", END)
REFLECTION_AGENT = reflection_graph.compile()



In [ ]:
from IPython.display import Image, display
display(Image(REFLECTION_AGENT.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
print("\n🪞 2. Reflection Agent") 
print("-" * 40)
reflection_result = REFLECTION_AGENT.invoke({"task": PUZZLE})
print(f"   Output: {reflection_result.content}...")

In [ ]:
# =============================================================================
# 🔄 3. REFLEXION AGENT (SUCCEEDS - Multiple iterations learn constraints)
# =============================================================================
class ReflexionState(TypedDict):
    task: str
    attempts: List[str]
    reflections: List[str]
    iteration: int
    result: str

def reflexion_act(state: ReflexionState) -> Dict:
    iteration = state.get("iteration", 0)
    context = "\n".join(state["reflections"][-2:]) if state["reflections"] else ""
    prompt = f"Reflexion #{iteration+1}: {state['task']}\nPast: {context}\nNEW ATTEMPT:"
    msg = llm.invoke([HumanMessage(content=prompt)])
    print(msg.content)
    return {"attempts": state["attempts"] + [msg.content], "iteration": iteration + 1}

def reflexion_reflect(state: ReflexionState) -> Dict:
    latest = state["attempts"][-1]
    reflection = llm.invoke([HumanMessage(f"{state['task']}\nAttempt: {latest}\nREFLECT: correct output?")])
    return {"reflections": state["reflections"] + [reflection.content]}

def reflexion_continue(state: ReflexionState) -> Literal["reflect", "end"]:
    return "end" if state["iteration"] >= 4 else "reflect"  # More iterations needed

reflexion_graph = StateGraph(ReflexionState)
reflexion_graph.add_node("act", reflexion_act)
reflexion_graph.add_node("reflect", reflexion_reflect)
reflexion_graph.add_edge(START, "act")
reflexion_graph.add_conditional_edges("act", reflexion_continue, {"reflect": "reflect", "end": END})
reflexion_graph.add_edge("reflect", "act")
REFLEXION_AGENT = reflexion_graph.compile()



In [ ]:
print("\n🔄 3. Reflexion Agent")
print("-" * 40)
reflexion_result = REFLEXION_AGENT.invoke({
        "task": PUZZLE, "attempts": [], "reflections": [], "iteration": 0, "result": ""
    })
print(f"   Output: {reflexion_result['reflections']}...")

In [ ]:
# =============================================================================
# 🌳 4. TREE SEARCH AGENT (SUCCEEDS - Systematic constraint checking)
# =============================================================================
def tree_explore(state: Dict) -> Dict:
    prompt = f"Tree Search: {PUZZLE}\nGenerate 4 complete crossing sequences checking ALL constraints each step."
    msg = llm.invoke([HumanMessage(content=prompt)])
    return {"paths": msg.content}

def tree_evaluate(state: Dict) -> Dict:
    prompt = f"Evaluate: {PUZZLE}\nPaths: {state['paths']}\nWhich satisfies ALL constraints? BEST SEQUENCE:"
    msg = llm.invoke([HumanMessage(content=prompt)])
    return {"result": msg.content, "status": "✅ SUCCESS (systematic)"}

tree_graph = StateGraph(dict)
tree_graph.add_node("explore", tree_explore)
tree_graph.add_node("evaluate", tree_evaluate)
tree_graph.add_edge(START, "explore")
tree_graph.add_edge("explore", "evaluate")
tree_graph.add_edge("evaluate", END)
TREE_AGENT = tree_graph.compile()



In [ ]:
print("\n🌳 4. Tree Search Agent")
print("-" * 40)
tree_result = TREE_AGENT.invoke({"task": PUZZLE})
print(f"   {tree_result['status']}")
print(f"   Output: {tree_result['result']}...")
    

In [ ]:
#!/usr/bin/env python3
"""
FIXED LangGraph REFLECTION Agent: Multiply 2 RANDOM 5-digit numbers
✅ Handles commas in numbers (ValueError fix)
"""

import os
import random
import re
from typing import Annotated, Dict, List, Literal
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

# Setup

# Generate RANDOM 5-digit numbers
num1 = random.randint(10000, 99999)
num2 = random.randint(10000, 99999)
CORRECT_ANSWER = num1 * num2
TASK = f"Multiply {num1} × {num2} = ? (Give exact answer)"

print(f"🔢 Numbers: {num1} × {num2}")
print(f"✅ Correct: {CORRECT_ANSWER:,}")
print("=" * 60)

# ✅ FIXED: Clean number extraction (handles commas/spaces)
def clean_number(text: str) -> int:
    """Extract & clean number from LLM output (handles 919,999,296 → 919999296)"""
    # Remove commas, spaces, $ signs, etc.
    cleaned = re.sub(r'[^\d]', '', str(text))
    return int(cleaned) if cleaned.isdigit() else 0

# =============================================================================
# REFLECTION STATE
# =============================================================================
class ReflectionState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    task: str
    attempts: List[str]
    critiques: List[str]
    iteration: int
    final_answer: str
    is_correct: bool

# =============================================================================
# FIXED NODES
# =============================================================================
def generate_attempt(state: ReflectionState) -> Dict:
    iteration = state.get("iteration", 0)
    past_critiques = "\n".join(state["critiques"][-2:]) if state["critiques"] else "No critiques"
    
    prompt = f"""MULTIPLICATION TASK (Iteration {iteration + 1}): {state['task']}

Past critiques:
{past_critiques}

CALCULATE EXACTLY: {num1} × {num2} = ?

ONLY the number (no commas needed):"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    attempt = response.content.strip()
    
    print(f"  📝 Attempt #{iteration + 1}: {attempt}")
    return {
        "attempts": state["attempts"] + [attempt],
        "iteration": iteration + 1,
        "messages": [AIMessage(content=f"Generated attempt #{iteration + 1}")]
    }

def critique_attempt(state: ReflectionState) -> Dict:
    latest_attempt = state["attempts"][-1]
    
    # ✅ FIXED: Use clean_number() to handle commas
    attempt_num = clean_number(latest_attempt)
    correct_num = CORRECT_ANSWER
    
    is_correct = abs(attempt_num - correct_num) <= 1  # Allow tiny rounding errors
    
    prompt = f"""CRITIQUE this multiplication:
{state['task']}
Attempt: {latest_attempt} → {attempt_num:,}
Correct: {correct_num:,}

Is it CORRECT? (Y/N)
ERROR: {'CORRECT' if is_correct else f'WRONG! Off by {correct_num - attempt_num:,}'}
IMPROVEMENT:"""
    
    critique = llm.invoke([HumanMessage(content=prompt)])
    critique_text = critique.content.strip()
    
    print(f"  🧐 Attempt value: {attempt_num:,}")
    print(f"  ✅ Correct? {'YES' if is_correct else 'NO'}")
    print(f"  📊 Error: {correct_num - attempt_num:,}")
    
    return {
        "critiques": state["critiques"] + [critique_text],
        "is_correct": is_correct,
        "messages": [AIMessage(content="Critique complete")]
    }

def should_continue(state: ReflectionState) -> Literal["continue", "end"]:
    max_iterations = 5
    return "end" if state["is_correct"] or state["iteration"] >= max_iterations else "continue"

def finalize(state: ReflectionState) -> Dict:
    final = state["attempts"][-1]
    final_num = clean_number(final)
    print(f"\n🎯 FINAL RESULT: {final} → {final_num:,}")
    print(f"✅ VERIFIED: {'CORRECT ✓' if state['is_correct'] else 'INCORRECT ✗'}")
    print(f"📊 Iterations: {state['iteration']}")
    print(f"🔍 Critiques: {len(state['critiques'])}")
    
    return {
        "final_answer": final,
        "messages": [AIMessage(content=f"FINAL: {final_num:,} ({'✓' if state['is_correct'] else '✗'})")]
    }

# =============================================================================
# BUILD GRAPH
# =============================================================================
workflow = StateGraph(ReflectionState)
workflow.add_node("generate", generate_attempt)
workflow.add_node("critique", critique_attempt)
workflow.add_node("finalize", finalize)

workflow.add_edge(START, "generate")
workflow.add_conditional_edges("generate", should_continue, {"continue": "critique", "end": "finalize"})
workflow.add_edge("critique", "generate")
workflow.add_edge("finalize", END)

reflection_agent = workflow.compile()

# =============================================================================
# RUN
# =============================================================================
if __name__ == "__main__":
    print("🤖 FIXED REFLECTION AGENT - NO MORE ValueError!")
    print(f"TASK: {TASK}")
    print("=" * 70)
    
    result = reflection_agent.invoke({
        "messages": [HumanMessage(content=TASK)],
        "task": TASK,
        "attempts": [],
        "critiques": [],
        "iteration": 0,
        "final_answer": "",
        "is_correct": False
    })
    
    print("\n" + "="*70)
    print("🎉 REFLECTION COMPLETE!")


In [ ]:
from IPython.display import Image, display
display(Image(reflection_agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
#!/usr/bin/env python3
"""
LangGraph REFLEXION Agent: Multiply 2 RANDOM 5-digit numbers
ACT → REFLECT → ACT → REFLECT → ... (Verbal Reinforcement Learning)
"""

import os
import random
import re
from typing import Annotated, Dict, List, Literal
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

# Setup

# Generate RANDOM 5-digit numbers
num1 = random.randint(10000, 99999)
num2 = random.randint(10000, 99999)
CORRECT_ANSWER = num1 * num2
TASK = f"Multiply {num1} × {num2} = ? (Give exact answer)"

print(f"🔢 Numbers: {num1} × {num2}")
print(f"✅ Correct: {CORRECT_ANSWER:,}")
print("=" * 60)

# Clean number extraction (handles commas)
def clean_number(text: str) -> int:
    cleaned = re.sub(r'[^\d]', '', str(text))
    return int(cleaned) if cleaned.isdigit() else 0

# =============================================================================
# REFLEXION STATE (Tracks verbal self-improvement)
# =============================================================================
class ReflexionState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    task: str
    trajectory: List[str]      # All attempts (ACT steps)
    reflections: List[str]     # Verbal self-critiques
    iteration: int
    final_answer: str
    is_correct: bool

# =============================================================================
# REFLEXION NODES (ACT → REFLECT loop)
# =============================================================================
def act_node(state: ReflexionState) -> Dict:
    """ACT: Generate next trajectory/attempt"""
    iteration = state.get("iteration", 0)
    
    # Verbal context from past reflections (key Reflexion feature)
    reflection_context = "\n".join([f"R{i+1}: {r[:80]}..." for i, r in enumerate(state["reflections"][-3:])])
    
    prompt = f"""REFLEXION AGENT - ITERATION #{iteration + 1}
TASK: {state['task']}

VERBAL SELF-REFLECTIONS (learn from mistakes):
{reflection_context if reflection_context else 'No reflections yet'}

ACT: Generate your BEST trajectory/calculation now.
Give ONLY the final number:"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    trajectory = response.content.strip()
    
    print(f"  🎯 ACT #{iteration + 1}: {trajectory}")
    return {
        "trajectory": state["trajectory"] + [trajectory],
        "iteration": iteration + 1,
        "messages": [AIMessage(content=f"ACT #{iteration + 1}: {trajectory}")]
    }

def reflect_node(state: ReflexionState) -> Dict:
    """REFLECT: Verbal self-critique (core Reflexion mechanism)"""
    latest_trajectory = state["trajectory"][-1]
    attempt_num = clean_number(latest_trajectory)
    correct_num = CORRECT_ANSWER
    is_correct = abs(attempt_num - correct_num) <= 1
    
    prompt = f"""REFLEXION: Verbal Self-Critique
TASK: {state['task']}
LATEST TRAJECTORY: {latest_trajectory} → {attempt_num:,}
CORRECT: {correct_num:,}

VERBAL REFLECTION:
1. Was this CORRECT? {is_correct}
2. What went WRONG? (digit errors, carrying mistakes, etc.)
3. How to IMPROVE next time?

Be VERBAL and SPECIFIC about calculation errors:"""
    
    reflection = llm.invoke([HumanMessage(content=prompt)])
    reflection_text = reflection.content.strip()
    
    print(f"  🧠 REFLECT #{len(state['reflections']) + 1}: {reflection_text[:100]}...")
    print(f"  ✅ Correct so far? {'YES' if is_correct else 'NO'}")
    
    return {
        "reflections": state["reflections"] + [reflection_text],
        "is_correct": is_correct,
        "messages": [AIMessage(content="REFLECT: Verbal self-improvement")]
    }

def should_continue(state: ReflexionState) -> Literal["reflect", "end"]:
    """Continue reflexion until correct OR max iterations"""
    max_iterations = 6
    return "end" if state["is_correct"] or state["iteration"] >= max_iterations else "reflect"

def finalize_node(state: ReflexionState) -> Dict:
    """Finalize with complete Reflexion trace"""
    final_trajectory = state["trajectory"][-1]
    final_num = clean_number(final_trajectory)
    
    print("\n" + "="*80)
    print("🎯 REFLEXION COMPLETE - VERBAL REINFORCEMENT LEARNING")
    print("="*80)
    print(f"TASK: {state['task']}")
    print(f"FINAL: {final_trajectory} → {final_num:,}")
    print(f"✅ STATUS: {'CORRECT ✓' if state['is_correct'] else 'INCORRECT ✗'}")
    print(f"📈 Iterations: {state['iteration']}")
    print(f"🧠 Reflections: {len(state['reflections'])}")
    print("\n📜 REFLEXION TRACE:")
    for i, (traj, refl) in enumerate(zip(state["trajectory"][-3:], state["reflections"][-3:]), 1):
        print(f"  {i}. ACT: {traj[:40]}... → REFLECT: {refl[:60]}...")
    
    return {
        "final_answer": final_trajectory,
        "messages": [AIMessage(content=f"REFLEXION SUCCESS: {final_num:,}")]
    }

# =============================================================================
# BUILD REFLEXION GRAPH
# =============================================================================
workflow = StateGraph(ReflexionState)

workflow.add_node("act", act_node)
workflow.add_node("reflect", reflect_node)
workflow.add_node("finalize", finalize_node)

# Reflexion loop: ACT → REFLECT → ACT → ...
workflow.add_edge(START, "act")
workflow.add_conditional_edges("act", should_continue, {"reflect": "reflect", "end": "finalize"})
workflow.add_edge("reflect", "act")
workflow.add_edge("finalize", END)

reflexion_agent = workflow.compile()

# =============================================================================
# RUN REFLEXION AGENT
# =============================================================================
if __name__ == "__main__":
    print("🔄 REFLEXION AGENT: Verbal Reinforcement Learning")
    print(f"TASK: {TASK}")
    print("=" * 70)
    
    result = reflexion_agent.invoke({
        "messages": [HumanMessage(content=TASK)],
        "task": TASK,
        "trajectory": [],
        "reflections": [],
        "iteration": 0,
        "final_answer": "",
        "is_correct": False
    })


In [ ]:
from IPython.display import Image, display
display(Image(reflexion_agent.get_graph(xray=True).draw_mermaid_png()))